<h1>Chapter 8 -  Object references, mutability and recycling </h1>

<h2>Variables are not boxes</h2>

 it’s better to think
 of them as labels attached to objects.

a = [1, 2, 3]
b = a
a.append(4)
b

 Variables are assigned to objects only after the objects are created.

In [2]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))


In [3]:
x = Gizmo()

Gizmo id: 10211728


In [4]:
y = Gizmo() * 10

Gizmo id: 35044424


<class 'TypeError'>: unsupported operand type(s) for *: 'Gizmo' and 'int'

First, Python tries to create a new Gizmo() object, so it prints the ID again.
But then Python tries to multiply it by 10 with *.
But there’s no rule for how to multiply a Gizmo object and a number, so Python throws this error:

In [5]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950} 

In [8]:
 charles = alex  

In [9]:
 alex == charles 

True

In [10]:
alex is not charles

False

<h2> Choosing between == and is</h2>
 The == operator compares the values of objects (the data they hold), while is compares
 their identities.
 We often care about values and not identities, so == appears more frequently than is in
 Python code. <br>
 By far, the most common case is checking whether a variable is bound to None. This is
 the recommended way to do it:
 x is None
 And the proper way to write its negation is:
 x is not None

<h2>The relative immutability of tuples</h2>
 Tuples are immutable, but…
Tuples themselves are immutable containers—you can’t change which objects they point to.
However, if a tuple contains a mutable object (like a list), the contents of that mutable object can be changed.

In [14]:
t1 = (1, 2, [30, 40])
t1[-1].append(99)  # This modifies the list inside the tuple


The tuple structure didn’t change (still points to the same list).

But the list’s content did—this is what’s called “relative immutability.”

<h2> Copies are shallow by default</h2>
A shallow copy creates a new container, but doesn’t copy the items inside it—it just copies their references.

In [16]:
import copy
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)  # or copy.copy(l1)


In [17]:
l2 == l1 

True

In [18]:
l2 is l1 

False

<h3>+= Behaves Differently for Lists and Tuples</h3>
For lists, += modifies the list in place.

For tuples, += creates a new tuple and reassigns the variable.

In [25]:
l2[2] += (10, 11)  # rebinds l2[2] to a new tuple, l1[2] stays the same
l2[1] += [33, 22]  # modifies list in place, affects both l1 and l2



In [26]:
print('l1:', l1)
print('l2:', l2)

l1: [3, [55, 44, 33, 22, 33, 22, 33, 22], (7, 8, 9)]
l2: [3, [55, 44, 33, 22, 33, 22, 33, 22], (7, 8, 9, 10, 11, 10, 11, 10, 11)]


In [27]:
import copy
deep_l = copy.deepcopy(l1)


<h2> Function parameters as references Function parameters as references</h2>

Function Parameters Are Passed by Sharing
Python uses call by sharing (also known as call by object-sharing).

This means that:

Function arguments are references (pointers) to the objects.

Inside the function, the parameters become aliases for those objects.

However, rebinding the alias (parameter) doesn’t affect the original reference in the caller’s scope.

In [28]:
def f(a, b):
    a += b
    return a


In [29]:
#For immutable types (like numbers, strings, tuples):
x = 1
y = 2
f(x, y)  # Returns 3, but x remains 1


3

In [30]:
#For mutable types (like lists):
a = [1, 2]
b = [3, 4]
f(a, b)  # Modifies a in-place → a becomes [1, 2, 3, 4]


[1, 2, 3, 4]

<h3>Mutable types as parameter defaults: bad idea</h3>
If a function/class receives mutable arguments, you should:

Make a copy if you don’t intend to modify the caller’s data.

Or clearly document that the input will be modified.

This avoids surprising behavior and follows the Principle of Least Astonishment.

In [37]:
class HauntedBus:
 """A bus model haunted by ghost passengers"""
 def __init__(self, passengers=[]):
     self.passengers = passengers
 def pick(self, name):
     self.passengers.append(name)
 def drop(self, name):
     self.passengers.remove(name)

In [39]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers


['Alice', 'Bill']

In [40]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers


['Bill', 'Charlie']

In [41]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers


['Carrie', 'Dave', 'Carrie']

In [42]:
bus3 = HauntedBus()
bus3.passengers


['Carrie', 'Dave', 'Carrie']

In [43]:
bus3.pick('Dave')
bus2.passengers


['Carrie', 'Dave', 'Carrie', 'Dave']

In [44]:
bus2.passengers is bus3.passengers
bus1.passengers

['Bill', 'Charlie']

basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

In [48]:
class TwilightBus:
 """A bus model that makes passengers vanish"""
 def __init__(self, passengers=None):
     if passengers is None:
         self.passengers = []
     else:
         self.passengers = passengers
 def pick(self, name):
     self.passengers.append(name)
 def drop(self, name):
     self.passengers.remove(name)

<h3>del Deletes Names, Not Objects:</h3>

The del keyword in Python doesn't directly destroy objects in memory. Instead, it removes a name (a variable) that was pointing to an object.
Think of a name like a sticker on a balloon (the object). del just peels off the sticker. The balloon is still there.

Garbage Collection Happens When Objects Are Unreachable:

Python has a process called "garbage collection" that reclaims memory used by objects that are no longer needed.
An object becomes a candidate for garbage collection when it's no longer reachable by any part of your program. This usually happens when:
The last variable holding a reference to it is deleted using del.
The last variable holding a reference to it is reassigned to something else (the "sticker" is moved to a different balloon or thrown away).
The object is part of a cycle of references with other objects that are also unreachable from the main program (like two balloons holding onto each other but nothing else holding onto them).

<h3>__del__ is for Cleanup, Not Destruction Control:</h3>
The Python interpreter calls __del__ (if it's defined in a class) just before an object is about to be garbage collected.
It's meant to give the object a chance to release external resources it might be holding onto (like closing files or network connections).
Using __del__ properly is tricky, and beginners often try to use it incorrectly. The text advises that you'll rarely need to implement __del__.


In [50]:
import weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye)
print(f"ender.alive: {ender.alive}")

del s1
print(f"ender.alive: {ender.alive}")

s2 = 'spam'
print(f"ender.alive: {ender.alive}")

ender.alive: True
ender.alive: True
Gone with the wind...
ender.alive: False


<h3>Weak References</h3> Strong References Keep Objects Alive: Normally, when you assign an object to a variable, you create a strong reference. As long as at least one strong reference to an object exists, the Python garbage collector will not destroy it.

Weak References Don't Prevent Garbage Collection: Weak references, on the other hand, are references to an object that do not prevent that object from being garbage collected if it's the only remaining reference. They are useful when you want to refer to an object but don't want to keep it alive unnecessarily.

 A common use case for weak references is in caching. You might want to store frequently used objects in a cache for faster access. However, you don't want these cached objects to live forever just because the cache holds a reference to them, especially if the main parts of your program are no longer using them. Weak references allow the cache to refer to these objects without preventing their garbage collection when memory is needed.

In [51]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
wref()


{0, 1}

In [52]:

a_set = {2, 3, 4}
wref()


{0, 1}


wref = weakref.ref(a_set): Creates a weak reference wref to the set referenced by a_set.
wref(): Calling wref() returns the original set {0, 1} because it's still alive (referenced by a_set). In the console, this result is also assigned to _.

<h4> Limitations:</h4>

Basic list and dict instances cannot be weak reference targets directly. However, the text provides a simple workaround: you can create a plain subclass of list or dict. Instances of these subclasses can be weakly referenced.

In [55]:
import weakref

# Trying to create a weak reference to a basic list will raise a TypeError
# my_list = [1, 2, 3]
# wref_list = weakref.ref(my_list) # This will fail

class MyList(list):
    """list subclass whose instances may be weakly referenced"""
    pass

a_list = MyList(range(10))
wref_to_a_list = weakref.ref(a_list) # This works
print(wref_to_a_list)

<weakref at 0x24df8c0; to 'MyList' at 0x24d8458>


In [56]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1


True

t[:] does not make a copy, but returns a
 reference to the same object.

In [57]:
t3 = t1[:]
t3 is t1


True

In [60]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)  # Creating a new tuple from scratch.
t3 is t1  # t1 and t3 are equal, but not the same object.


False

In [61]:
s1 = 'ABC'
s2 = 'ABC'  # Creating a second str from scratch.
s2 is s1 # Surprise: a and b refer to the same str!


True